In [2]:
!pip install transformers

In [24]:
!pip install rouge

In [25]:
!pip install rouge-score

In [125]:
!pip install transformers[sentencepiece]

     |████████████████████████████████| 1.2MB 1.8MB/s eta 0:00:01


In [8]:
!pip install sentencepiece

In [4]:
!pip install ohmeow-blurr -q
!pip install bert-score -q

In [11]:
!pip install blurr

     ---------------------------------------- 52.9/52.9 kB 1.3 MB/s eta 0:00:00
     -------------------------------------- 132.5/132.5 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 8.8/8.8 MB 35.0 MB/s eta 0:00:00
     ---------------------------------------- 79.5/79.5 kB 4.6 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=203683717b8706bfd54e158e787ec5d90c4c16433a58892ae2094d3e27f72f18
  Stored in directory: c:\users\pc\appdata\local\pip\cache\wheels\70\4a\46\1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


In [5]:
!pip install sacremoses

  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=2653e8a2a741db1952812f3e5e2055a4c0fd090349cb20a6680ff4d9a2801f70
  Stored in directory: c:\users\pc\appdata\local\pip\cache\wheels\00\24\97\a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


In [4]:
!pip install tabulate

In [29]:
!pip install wandb

  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21503 sha256=d1eae4fa731cceacf0fa535a2c14291482c1d239fe949982a57fdab22e49b59f
  Stored in directory: c:\users\pc\appdata\local\pip\cache\wheels\54\4e\28\3ed0e1c8a752867445bab994d2340724928aa3ab059c57c8db
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=01d9fa4064a9ca6e3b3116769be472cc06c31361e396ca902fb62bee44db9017
  Stored in directory: c:\users\pc\appdata\local\pip\cache\wheels\e7\f3\22\152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built promise pathtools


In [7]:
!pip install datasets

In [10]:
!pip install nltk

In [22]:
!pip install absl-py

Note: you may need to restart the kernel to use updated packages.


## Import requirements

In [1]:
import torch
print(torch.cuda.is_available())

True


In [4]:
from transformers import pipeline
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer
from transformers import SummarizationPipeline
import re
from rouge import Rouge
import pickle
import numpy as np
import json

In [4]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
import numpy as np
import datasets

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)

from tabulate import tabulate
import nltk
from datetime import datetime
from datasets import load_dataset

In [3]:
import os

In [4]:
from transformers import MBartForConditionalGeneration, MBartTokenizer

# Milestone 3

## Import Multinews Datasets

In [2]:
with open('multinews_dataset/test_articles.txt', encoding="utf8") as f:
    multinews_articles = f.readlines()

In [3]:
with open('multinews_dataset/test_summ.txt', encoding="utf8") as f:
    multinews_summ = f.readlines()

## Clean datasets

In [4]:
def clean_article(article):
    #keep only alphanumeric values
    alphanum_article = re.sub(rf'[^a-z0-9 \'\.]', ' ', article)
    #delete unwanted white spaces
    whitespace_mid = re.sub(rf'\b( +)\b', ' ', alphanum_article)
    whitespace_end = re.sub(rf'( +)$', '', whitespace_mid)
    whitespace_beg = re.sub(rf'^( +)', '', whitespace_end)
    
    return whitespace_beg

In [5]:
def postproc_article(article):
    return re.sub(rf'(\.)', '', article)

In [22]:
rouge = Rouge()

## Get mean scores

In [47]:
def get_mean_scores(scores):
    rouge1_r = []
    rouge1_p = []
    rouge1_f = []
    rouge2_r = []
    rouge2_p = []
    rouge2_f = []
    rougel_r = []
    rougel_p = []
    rougel_f = []
    for score in scores:
        rouge1_r.append(score[0]['rouge-1']['r'])
        rouge1_p.append(score[0]['rouge-1']['p'])
        rouge1_f.append(score[0]['rouge-1']['f'])
        
        rouge2_r.append(score[0]['rouge-2']['r'])
        rouge2_p.append(score[0]['rouge-2']['p'])
        rouge2_f.append(score[0]['rouge-2']['f'])
        
        rougel_r.append(score[0]['rouge-l']['r'])
        rougel_p.append(score[0]['rouge-l']['p'])
        rougel_f.append(score[0]['rouge-l']['f'])
    
    results = [np.mean(rouge1_r), np.mean(rouge1_p), np.mean(rouge1_f),
               np.mean(rouge2_r), np.mean(rouge2_p), np.mean(rouge2_f),
               np.mean(rougel_r), np.mean(rougel_p), np.mean(rougel_f)]
    
    return results

## Define BART Model

In [71]:
summarizer = pipeline('summarization', model='facebook/bart-large-cnn', tokenizer='facebook/bart-large-cnn')

## BART on Multinews

In [95]:
%%time
scores_bart_multinews = []
for idx, article in enumerate(multinews_articles[:5000]):
    if idx % 100 == 0:
        print(idx)
    generated_summary = summarizer(article, min_length = 50, max_length = 250, do_sample=False)
    gen_summ = postproc_article(
                   clean_article(
                       generated_summary[0]['summary_text'].lower()))
    
    target_summ = postproc_article(
                      clean_article(multinews_summ[idx]))
    
    scores_bart_multinews.append(rouge.get_scores(gen_summ, target_summ))    

0


Your max_length is set to 250, but you input_length is only 149. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=74)
Your max_length is set to 250, but you input_length is only 236. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=118)


100


Your max_length is set to 250, but you input_length is only 230. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=115)
Your max_length is set to 250, but you input_length is only 224. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=112)


200


Token indices sequence length is longer than the specified maximum sequence length for this model (2272 > 1024). Running this sequence through the model will result in indexing errors


IndexError: index out of range in self

In [52]:
results_bart_multinews = get_mean_scores(scores_bart_multinews)

In [53]:
results_bart_multinews

[0.19398086374359405,
 0.566394042529543,
 0.2837468827696245,
 0.06462788884844443,
 0.2296533666107905,
 0.09864009538725348,
 0.12560604791652127,
 0.36465727716591695,
 0.18340505161625237]

## BART on NewsRoom

In [62]:
newsroom_dataset = []
with open('newsroom_dataset/test.jsonl', 'r') as json_file:
    json_list = list(json_file)

for json_str in json_list:
    newsroom_dataset.append(json.loads(json_str))

In [85]:
%%time
scores_bart_newsroom = []
for idx, article in enumerate(newsroom_dataset[:5000]):
    if len(article['text']) < 5000:
        print(idx)
        generated_summary = summarizer(article['text'].lower(), min_length = 50, max_length = 250, do_sample=False)
        gen_summ = postproc_article(
                   clean_article(
                       generated_summary[0]['summary_text'].lower()))
    
        target_summ = postproc_article(
                      clean_article(article['summary'].lower()))
    
        scores_bart_newsroom.append(rouge.get_scores(gen_summ, target_summ))    

0
1
2


Your max_length is set to 250, but you input_length is only 74. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=37)


3
4
6


Your max_length is set to 250, but you input_length is only 70. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=35)


8
11
12
13
17
19
20
21
22
23
24
25
26
28
31
32
33
34
37
40
41
42
43
44
45
46
47
49
50
51
52
53
54
55
56
57
58


...

248
250
251
253
254
255
256
257
258
260
261
263
264
266
267
268
269
270
272
273
274
275
277
279
281
283
284
287
289
290


Your max_length is set to 250, but you input_length is only 184. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=92)


291
293
294
295
296


Your max_length is set to 250, but you input_length is only 237. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=118)


297
299
CPU times: user 44min 6s, sys: 5min 19s, total: 49min 25s
Wall time: 45min 26s


In [95]:
with open('results/bart_newsroom', 'wb') as fp:
    pickle.dump(scores_bart_newsroom, fp)

In [87]:
results_bart_newsroom = get_mean_scores(scores_bart_newsroom)

In [88]:
results_bart_newsroom

[0.3886622495938386,
 0.2133014910052308,
 0.25999745918996875,
 0.17274045629097223,
 0.10011144012944892,
 0.11693256036895888,
 0.3177664210377903,
 0.17658347124434656,
 0.21367429518976228]

## Define MBART Model

In [11]:
loaded_tokenizer = AutoTokenizer.from_pretrained("ahmeddbahaa/mbart-large-50-finetuned-ar-wikilingua")
loaded_model = TFAutoModelForSeq2SeqLM.from_pretrained("ahmeddbahaa/mbart-large-50-finetuned-ar-wikilingua", from_pt = True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFMBartForConditionalGeneration: ['lm_head.weight']
- This IS expected if you are initializing TFMBartForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFMBartForConditionalGeneration from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFMBartForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMBartForConditionalGeneration for predictions without further training.


In [19]:
mbart = SummarizationPipeline(model=loaded_model, tokenizer=tokenizer)

## MBART on Multinews

In [21]:
%%time
scores_mbart_multinews2 = []
for idx, article in enumerate(multinews_articles[:5000]):
    print(idx)
    generated_summary = mbart(multinews_articles[idx].lower(), min_length = 50, max_length = 250, do_sample = False)
    gen_summ = postproc_article(
                   clean_article(
                       generated_summary[0]['summary_text'].lower()))
    
    target_summ = postproc_article(
                      clean_article(multinews_summ[idx]))
    
    scores_mbart_multinews2.append(rouge.get_scores(gen_summ, target_summ)) 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34


Your max_length is set to 250, but you input_length is only 169. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=84)


35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80


Your max_length is set to 250, but you input_length is only 247. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=123)


81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116


ValueError: Hypothesis is empty.

In [25]:
%%time
scores_mbart_multinews2 = []
for idx, article in enumerate(multinews_articles[:5000]):
    generated_summary = mbart(multinews_articles[idx].lower(), min_length = 50, max_length = 250, do_sample = False)
    gen_summ = postproc_article(
                   clean_article(
                       generated_summary[0]['summary_text'].lower()))
    
    target_summ = postproc_article(
                      clean_article(multinews_summ[idx]))
    
    scores_mbart_multinews2.append(rouge.get_scores(gen_summ, target_summ)) 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34


KeyboardInterrupt: 

In [26]:
scores_mbart_multinews.append(scores_mbart_multinews2)

In [34]:
for score in scores_mbart_multinews2:
    scores_mbart_multinews.append(score)

In [36]:
with open('results/mbart_multinews', 'wb') as fp:
    pickle.dump(scores_mbart_multinews, fp)

In [48]:
results_mbart_multinews = get_mean_scores(scores_mbart_multinews)

In [49]:
results_mbart_multinews

[0.15190895015885073,
 0.48878609433962067,
 0.22636914634400293,
 0.038035467795156756,
 0.14922600079869042,
 0.05854957294613635,
 0.09400774399952658,
 0.3040036051022214,
 0.1401548530546241]

## MBART on NewsRoom

In [89]:
%%time
scores_mbart_newsroom = []
for idx, article in enumerate(newsroom_dataset[:5000]):
    if len(article['text']) < 4000:
        print(idx)
        generated_summary = mbart(article['text'].lower(), min_length = 50, max_length = 250, do_sample=False)
        gen_summ = postproc_article(
                   clean_article(
                       generated_summary[0]['summary_text'].lower()))
    
        target_summ = postproc_article(
                      clean_article(article['summary'].lower()))
    
        scores_mbart_newsroom.append(rouge.get_scores(gen_summ, target_summ))    

0
1
2


Your max_length is set to 250, but you input_length is only 87. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=43)


3
4
6


Your max_length is set to 250, but you input_length is only 81. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=40)


8
11
12
13
17
19
20
21
22
23
24
25
26
28
31
32
33
34
37
40
41
42
43
44
45
46
47
49
50
51
52
53
54
55
56
57
58
59


...

Your max_length is set to 250, but you input_length is only 195. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=97)


291
293
294
295
296


Your max_length is set to 250, but you input_length is only 247. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=123)


297
299
300


KeyboardInterrupt: 

In [96]:
with open('results/mbart_newsroom', 'wb') as fp:
    pickle.dump(scores_mbart_newsroom, fp)

In [93]:
results_mbart_newsroom = get_mean_scores(scores_mbart_newsroom)

In [94]:
results_mbart_newsroom

[0.42353486766787923,
 0.26831545664820483,
 0.3039959184876281,
 0.2283095711836132,
 0.15472137820969037,
 0.16788760038538053,
 0.3624112544416273,
 0.23556015145056014,
 0.26331424199200837]

## Demo

In [99]:
demo_tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-cnn')
demo_model = TFAutoModelForSeq2SeqLM.from_pretrained('facebook/bart-large-cnn', from_pt = True)

All PyTorch model weights were used when initializing TFBartForConditionalGeneration.

Some weights or buffers of the TF 2.0 model TFBartForConditionalGeneration were not initialized from the PyTorch model and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [100]:
multinews_articles[0]

"gop eyes gains as voters in 11 states pick governors enlarge this image toggle caption jim cole/ap jim cole/ap voters in 11 states will pick their governors tonight , and republicans appear on track to increase their numbers by at least one , with the potential to extend their hold to more than two-thirds of the nation ' s top state offices . eight of the gubernatorial seats up for grabs are now held by democrats ; three are in republican hands . republicans currently hold 29 governorships , democrats have 20 , and rhode island ' s gov . lincoln chafee is an independent . polls and race analysts suggest that only three of tonight ' s contests are considered competitive , all in states where incumbent democratic governors aren ' t running again : montana , new hampshire and washington . while those state races remain too close to call , republicans are expected to wrest the north carolina governorship from democratic control , and to easily win gop-held seats in utah , north dakota and

In [102]:
demo_tokenizer(clean_article(multinews_articles[0]))

{'input_ids': [0, 571, 1517, 2473, 3077, 25, 1983, 11, 365, 982, 1339, 12066, 22368, 42, 2274, 23012, 3747, 1236, 757, 1029, 459, 6256, 1236, 757, 1029, 459, 6256, 1983, 11, 365, 982, 40, 1339, 49, 12066, 3422, 8, 16441, 1253, 2082, 15, 1349, 7, 712, 49, 1530, 30, 23, 513, 65, 19, 5, 801, 7, 4442, 49, 946, 7, 55, 87, 80, 29193, 9, 5, 1226, 128, 579, 299, 194, 4088, 479, 799, 9, 5, 18932, 3202, 62, 13, 17565, 32, 122, 547, 30, 38538, 2923, 130, 32, 11, 37958, 1420, 479, 16441, 1253, 855, 946, 1132, 12066, 7903, 38538, 2923, 33, 291, 8, 16448, 4636, 2946, 128, 579, 213, 705, 479, 784, 37654, 1855, 2001, 1942, 16, 41, 2222, 479, 4583, 8, 1015, 1066, 3608, 14, 129, 130, 9, 3422, 128, 579, 11997, 32, 1687, 2695, 70, 11, 982, 147, 9861, 7368, 12066, 2025, 128, 326, 878, 456, 27121, 1113, 92, 1368, 3914, 9959, 8, 14784, 1054, 479, 150, 167, 194, 4694, 1091, 350, 593, 7, 486, 16441, 1253, 32, 421, 7, 22160, 5, 1926, 512, 27343, 11738, 11887, 31, 7368, 797, 8, 7, 2773, 339, 821, 1517, 547, 3202

In [103]:
summarizer = pipeline('summarization', model='facebook/bart-large-cnn', tokenizer='facebook/bart-large-cnn')

In [104]:
summarizer(clean_article(multinews_articles[0]), min_length = 50, max_length = 250)

[{'summary_text': "Voters in 11 states will pick their governors tonight. Eight of the gubernatorial seats up for grabs are now held by democrats three are in republican hands. republicans currently hold 29 governorships democrats have 20 and rhode island ' s gov . lincoln chafee is an independent."}]

 # Final milestone

## Finetuning

In [4]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples["document"],
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str

### Load models

In [5]:
language = "english"

In [6]:
model_name = "facebook/bart-large-cnn"

In [6]:
model_name = "ahmeddbahaa/mbart-large-50-finetuned-ar-wikilingua"

In [16]:
model_name = "facebook/mbart-large-50"

In [8]:
model_name = "facebook/mbart-large-cc25"

In [19]:
model_name = "sshleifer/tiny-mbart"

In [5]:
model_name = "facebook/mbart-large-en-ro"

In [7]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [8]:
encoder_max_length = 256
decoder_max_length = 64

In [9]:
tokenizer = MBartTokenizer.from_pretrained(model_name, src_lang="en_XX", tgt_lang="en_XX")

In [10]:
model = MBartForConditionalGeneration.from_pretrained(model_name)

### Load datasets

In [9]:
# multinews dataset found in datasets module
data = datasets.load_dataset("multi_news", name=language, split="validation")

Using custom data configuration english
Reusing dataset multi_news (C:\Users\pc\.cache\huggingface\datasets\multi_news\english\1.0.0\04f6b32c1fe246d12beab94a36e0752c3ee43e024afb9edd1e6bcc87e1457b45)


In [10]:
data

Dataset({
    features: ['document', 'summary'],
    num_rows: 5622
})

In [9]:
#newshead dataset imported from local file
data = load_dataset('json', data_files = 'newsroom_dataset/test.jsonl')

Using custom data configuration default-6bde83e8a6e31240
Reusing dataset json (C:\Users\pc\.cache\huggingface\datasets\json\default-6bde83e8a6e31240\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
data

DatasetDict({
    train: Dataset({
        features: ['url', 'archive', 'title', 'date', 'text', 'summary', 'compression', 'coverage', 'density', 'compression_bin', 'coverage_bin', 'density_bin'],
        num_rows: 108862
    })
})

In [11]:
dataset = data['train']

### Train/Test sets utills

In [12]:
def flatten(example):
    return {
        "document": example["text"],
        "summary": example["title"]
    }


def list2samples(example):
    documents = []
    summaries = []
    for sample in zip(example["document"], example["summary"]):
        if len(sample[0]) > 0:
            documents += sample[0]
            summaries += sample[1]
    return {"document": documents, "summary": summaries}


dataset = dataset.map(flatten, remove_columns = ['url', 'archive', 'date',
                                               'compression', 'coverage', 'density', 
                                               'compression_bin', 'coverage_bin', 'density_bin'])
#dataset = data.map(list2samples, batched=True)

train_data_txt, validation_data_txt = dataset.train_test_split(test_size=0.05).values()

Loading cached processed dataset at C:\Users\pc\.cache\huggingface\datasets\json\default-6bde83e8a6e31240\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b\cache-f018e50869e957ea.arrow


In [13]:
validation_data_txt

Dataset({
    features: ['title', 'text', 'summary', 'document'],
    num_rows: 5444
})

In [14]:
all_data = validation_data_txt.map(flatten, remove_columns = ['summary', 'document'])
#summary

  0%|          | 0/5444 [00:00<?, ?ex/s]

In [15]:
train_data_txt, validation_data_txt = all_data.train_test_split(test_size=0.1).values()

In [16]:
def batch_tokenize_preprocess(batch, tokenizer, max_source_length, max_target_length):
    #source, target = batch["document"], batch["summary"]
    #source, target = batch["text"], batch["summary"]
    source, target = batch["text"], batch["title"]
    source_tokenized = tokenizer(
        source, padding="max_length", truncation=True, max_length=max_source_length
    )
    target_tokenized = tokenizer(
        target, padding="max_length", truncation=True, max_length=max_target_length
    )

    batch = {k: v for k, v in source_tokenized.items()}
    # Ignore padding in the loss
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in target_tokenized["input_ids"]
    ]
    return batch


train_data = train_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=train_data_txt.column_names,
)

validation_data = validation_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=validation_data_txt.column_names,
)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [17]:
nltk.download("punkt", quiet=True)

metric = datasets.load_metric("rouge")


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results from ROUGE
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 1) for k, v in result.items()}
    return result

In [18]:
training_args = Seq2SeqTrainingArguments(
    output_dir="results",
    num_train_epochs=1, 
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=1,  
    per_device_eval_batch_size=1,
    # learning_rate=3e-05,
    warmup_steps=500,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True,
    logging_dir="logs",
    logging_steps=50,
    save_total_limit=3,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=validation_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

## BART on Multinews

In [14]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 563
  Batch size = 4


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: mhakan20. Use `wandb login --relogin` to force relogin


{'eval_loss': 4.3367719650268555,
 'eval_rouge1': 30.8261,
 'eval_rouge2': 8.2431,
 'eval_rougeL': 18.5757,
 'eval_rougeLsum': 25.6941,
 'eval_gen_len': 68.7709,
 'eval_runtime': 290.3749,
 'eval_samples_per_second': 1.939,
 'eval_steps_per_second': 0.486}

In [20]:
torch.cuda.empty_cache()

In [14]:
trainer.train()

C:\Users\pc\miniconda3\envs\pytorch\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5059
  Num Epochs = 5
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 12650
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: mhakan20. Use `wandb login --relogin` to force relogin


Step,Training Loss
50,4.235300
100,3.961200
150,3.775000
200,3.837900
250,3.900600
300,3.747100
350,3.802700
400,3.783600
450,3.755100
500,3.891500


Saving model checkpoint to results\checkpoint-500
Configuration saved in results\checkpoint-500\config.json
Model weights saved in results\checkpoint-500\pytorch_model.bin
tokenizer config file saved in results\checkpoint-500\tokenizer_config.json
Special tokens file saved in results\checkpoint-500\special_tokens_map.json
Saving model checkpoint to results\checkpoint-1000
Configuration saved in results\checkpoint-1000\config.json
Model weights saved in results\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in results\checkpoint-1000\tokenizer_config.json
Special tokens file saved in results\checkpoint-1000\special_tokens_map.json
Saving model checkpoint to results\checkpoint-1500
Configuration saved in results\checkpoint-1500\config.json
Model weights saved in results\checkpoint-1500\pytorch_model.bin
tokenizer config file saved in results\checkpoint-1500\tokenizer_config.json
Special tokens file saved in results\checkpoint-1500\special_tokens_map.json
Saving model check

Special tokens file saved in results\checkpoint-10500\special_tokens_map.json
Deleting older checkpoint [results\checkpoint-9000] due to args.save_total_limit
Saving model checkpoint to results\checkpoint-11000
Configuration saved in results\checkpoint-11000\config.json
Model weights saved in results\checkpoint-11000\pytorch_model.bin
tokenizer config file saved in results\checkpoint-11000\tokenizer_config.json
Special tokens file saved in results\checkpoint-11000\special_tokens_map.json
Deleting older checkpoint [results\checkpoint-9500] due to args.save_total_limit
Saving model checkpoint to results\checkpoint-11500
Configuration saved in results\checkpoint-11500\config.json
Model weights saved in results\checkpoint-11500\pytorch_model.bin
tokenizer config file saved in results\checkpoint-11500\tokenizer_config.json
Special tokens file saved in results\checkpoint-11500\special_tokens_map.json
Deleting older checkpoint [results\checkpoint-10000] due to args.save_total_limit
Saving mod

TrainOutput(global_step=12650, training_loss=2.729535662986544, metrics={'train_runtime': 7511.7199, 'train_samples_per_second': 3.367, 'train_steps_per_second': 1.684, 'total_flos': 1.370422772760576e+16, 'train_loss': 2.729535662986544, 'epoch': 5.0})

In [15]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 563
  Batch size = 2


{'eval_loss': 4.191427707672119,
 'eval_rouge1': 36.42,
 'eval_rouge2': 11.91,
 'eval_rougeL': 22.72,
 'eval_rougeLsum': 29.01,
 'eval_gen_len': 65.0,
 'eval_runtime': 404.8775,
 'eval_samples_per_second': 1.391,
 'eval_steps_per_second': 0.697,
 'epoch': 5.0}

In [19]:
model_before_tuning = AutoModelForSeq2SeqLM.from_pretrained(model_name)

test_samples = validation_data_txt[0] 

summaries_before_tuning = generate_summary(test_samples, model_before_tuning)[1]
summaries_after_tuning = generate_summary(test_samples, model)[1]

loading configuration file https://huggingface.co/facebook/bart-large-cnn/resolve/main/config.json from cache at C:\Users\pc/.cache\huggingface\transformers\199ab6c0f28e763098fd3ea09fd68a0928bb297d0f76b9f3375e8a1d652748f9.930264180d256e6fe8e4ba6a728dd80e969493c23d4caa0a6f943614c52d34ab
Model config BartConfig {
  "_name_or_path": "facebook/bart-large-cnn",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "force_bos_token_to_

In [30]:
print(test_samples['document'])

As the crude crept closer to Florida, the risky effort to contain the nation's worst oil spill hit a snag Wednesday when a diamond-edged saw became stuck in a thick pipe on a blown-out well at the bottom of the Gulf. 
  
 Coast Guard Adm. Thad Allen said the goal was to free the saw and finish the cut later in the day. This is the latest attempt to contain _ not plug _ the gusher. The best chance at stopping the leak is a relief well, which is at least two months from completion. 
  
 "I don't think the issue is whether or not we can make the second cut. It's about how fine we can make it, how smooth we can make it," Allen said. 
  
 If the cut is not as smooth as engineers would like, they would be forced to put a looser fitting cap on top of the oil spewing out. This cut-and-cap effort could temporarily increase the flow of oil by as much as 20 percent, though Allen said officials wouldn't know whether that had happened until the cut could be completed. 
  
 Engineers may have to bri

In [25]:
print(test_samples['summary'])

– British Petroleum CEO Tony Hayward has been roundly bashed for his unfortunate "I'd like my life back" comment—lots of fishermen and coastal residents want their lives back, too, not to mention relatives of the 11 men killed in the initial explosion. Today, Hayward apologized, reports the Huffington Post. "I made a hurtful and thoughtless comment," he said. "When I read that recently, I was appalled. I apologize, especially to the families of the 11 men who lost their lives in this tragic accident." His statement continues: "Those words don't represent how I feel about this tragedy, and certainly don't represent the hearts of the people of BP. My first priority is doing all we can to restore the lives of the people of the Gulf region and their families—to restore their lives, not mine." In the meantime, that saw remains stuck, notes AP.


In [23]:
summaries_before_tuning

['A diamond-edged saw becomes stuck in a thick pipe on a blown-out well. Coast Guard Adm. Thad Allen says the goal is to free the saw and finish the cut later in the day. If the cut is not as smooth as engineers would like, they would be forced to put a looser fitting cap.']

In [24]:
summaries_after_tuning

["– The Coast Guard is using a diamond-edged saw to cut through a thick pipe holding oil in the Gulf of Mexico, but it's a risky move that may backfire on them, reports the Los Angeles Times. Engineers hope to finish the cut today, but a second saw is needed to cut"]

## BART on Newshead

In [57]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 545
  Batch size = 2


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: mhakan20. Use `wandb login --relogin` to force relogin


{'eval_loss': 3.369145154953003,
 'eval_rouge1': 26.3,
 'eval_rouge2': 11.27,
 'eval_rougeL': 20.15,
 'eval_rougeLsum': 22.98,
 'eval_gen_len': 67.63,
 'eval_runtime': 447.8973,
 'eval_samples_per_second': 1.217,
 'eval_steps_per_second': 0.61}

In [58]:
trainer.train()

C:\Users\pc\miniconda3\envs\pytorch\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4899
  Num Epochs = 5
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 12250


Step,Training Loss
50,3.196500
100,3.120500
150,2.958400
200,2.930000
250,3.174100
300,3.179800
350,3.132700
400,3.330600
450,3.164700
500,3.294900


Saving model checkpoint to results\checkpoint-500
Configuration saved in results\checkpoint-500\config.json
Model weights saved in results\checkpoint-500\pytorch_model.bin
tokenizer config file saved in results\checkpoint-500\tokenizer_config.json
Special tokens file saved in results\checkpoint-500\special_tokens_map.json
Deleting older checkpoint [results\checkpoint-7000] due to args.save_total_limit
Saving model checkpoint to results\checkpoint-1000
Configuration saved in results\checkpoint-1000\config.json
Model weights saved in results\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in results\checkpoint-1000\tokenizer_config.json
Special tokens file saved in results\checkpoint-1000\special_tokens_map.json
Deleting older checkpoint [results\checkpoint-7500] due to args.save_total_limit
Saving model checkpoint to results\checkpoint-1500
Configuration saved in results\checkpoint-1500\config.json
Model weights saved in results\checkpoint-1500\pytorch_model.bin
tokenizer 

Saving model checkpoint to results\checkpoint-10500
Configuration saved in results\checkpoint-10500\config.json
Model weights saved in results\checkpoint-10500\pytorch_model.bin
tokenizer config file saved in results\checkpoint-10500\tokenizer_config.json
Special tokens file saved in results\checkpoint-10500\special_tokens_map.json
Deleting older checkpoint [results\checkpoint-9000] due to args.save_total_limit
Saving model checkpoint to results\checkpoint-11000
Configuration saved in results\checkpoint-11000\config.json
Model weights saved in results\checkpoint-11000\pytorch_model.bin
tokenizer config file saved in results\checkpoint-11000\tokenizer_config.json
Special tokens file saved in results\checkpoint-11000\special_tokens_map.json
Deleting older checkpoint [results\checkpoint-9500] due to args.save_total_limit
Saving model checkpoint to results\checkpoint-11500
Configuration saved in results\checkpoint-11500\config.json
Model weights saved in results\checkpoint-11500\pytorch_mo

TrainOutput(global_step=12250, training_loss=2.265575162303691, metrics={'train_runtime': 7108.1677, 'train_samples_per_second': 3.446, 'train_steps_per_second': 1.723, 'total_flos': 1.327080680718336e+16, 'train_loss': 2.265575162303691, 'epoch': 5.0})

In [59]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 545
  Batch size = 2


{'eval_loss': 3.370157480239868,
 'eval_rouge1': 38.04,
 'eval_rouge2': 26.09,
 'eval_rougeL': 33.09,
 'eval_rougeLsum': 34.85,
 'eval_gen_len': 64.29,
 'eval_runtime': 408.5014,
 'eval_samples_per_second': 1.334,
 'eval_steps_per_second': 0.668,
 'epoch': 5.0}

## BART on Newshead titles

In [27]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 545
  Batch size = 2


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: mhakan20. Use `wandb login --relogin` to force relogin


{'eval_loss': 4.67332124710083,
 'eval_rouge1': 12.69,
 'eval_rouge2': 3.67,
 'eval_rougeL': 10.41,
 'eval_rougeLsum': 11.51,
 'eval_gen_len': 68.17,
 'eval_runtime': 452.7467,
 'eval_samples_per_second': 1.204,
 'eval_steps_per_second': 0.603}

In [28]:
trainer.train()

C:\Users\pc\miniconda3\envs\pytorch\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4899
  Num Epochs = 5
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 12250


Step,Training Loss
50,4.486700
100,3.582800
150,3.763700
200,3.770700
250,3.838600
300,3.879800
350,3.848300
400,3.945400
450,4.078900
500,3.938400


Saving model checkpoint to results\checkpoint-500
Configuration saved in results\checkpoint-500\config.json
Model weights saved in results\checkpoint-500\pytorch_model.bin
tokenizer config file saved in results\checkpoint-500\tokenizer_config.json
Special tokens file saved in results\checkpoint-500\special_tokens_map.json
Deleting older checkpoint [results\checkpoint-11000] due to args.save_total_limit
Saving model checkpoint to results\checkpoint-1000
Configuration saved in results\checkpoint-1000\config.json
Model weights saved in results\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in results\checkpoint-1000\tokenizer_config.json
Special tokens file saved in results\checkpoint-1000\special_tokens_map.json
Deleting older checkpoint [results\checkpoint-11500] due to args.save_total_limit
Saving model checkpoint to results\checkpoint-1500
Configuration saved in results\checkpoint-1500\config.json
Model weights saved in results\checkpoint-1500\pytorch_model.bin
tokenize

Saving model checkpoint to results\checkpoint-10500
Configuration saved in results\checkpoint-10500\config.json
Model weights saved in results\checkpoint-10500\pytorch_model.bin
tokenizer config file saved in results\checkpoint-10500\tokenizer_config.json
Special tokens file saved in results\checkpoint-10500\special_tokens_map.json
Deleting older checkpoint [results\checkpoint-9000] due to args.save_total_limit
Saving model checkpoint to results\checkpoint-11000
Configuration saved in results\checkpoint-11000\config.json
Model weights saved in results\checkpoint-11000\pytorch_model.bin
tokenizer config file saved in results\checkpoint-11000\tokenizer_config.json
Special tokens file saved in results\checkpoint-11000\special_tokens_map.json
Deleting older checkpoint [results\checkpoint-9500] due to args.save_total_limit
Saving model checkpoint to results\checkpoint-11500
Configuration saved in results\checkpoint-11500\config.json
Model weights saved in results\checkpoint-11500\pytorch_mo

TrainOutput(global_step=12250, training_loss=2.590498660340601, metrics={'train_runtime': 7162.7091, 'train_samples_per_second': 3.42, 'train_steps_per_second': 1.71, 'total_flos': 1.327080680718336e+16, 'train_loss': 2.590498660340601, 'epoch': 5.0})

In [29]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 545
  Batch size = 2


{'eval_loss': 4.355747699737549,
 'eval_rouge1': 19.1,
 'eval_rouge2': 7.04,
 'eval_rougeL': 15.84,
 'eval_rougeLsum': 16.17,
 'eval_gen_len': 59.11,
 'eval_runtime': 369.8127,
 'eval_samples_per_second': 1.474,
 'eval_steps_per_second': 0.738,
 'epoch': 5.0}

## MBART on Multinews

In [22]:
test_samples = validation_data_txt[0] 

summaries_before_tuning = generate_summary(test_samples, model)[1]

In [15]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 563
  Batch size = 1


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: mhakan20. Use `wandb login --relogin` to force relogin


{'eval_loss': 5.407193660736084,
 'eval_rouge1': 20.7,
 'eval_rouge2': 5.9,
 'eval_rougeL': 14.2,
 'eval_rougeLsum': 16.1,
 'eval_gen_len': 37.2,
 'eval_runtime': 571.7055,
 'eval_samples_per_second': 0.985,
 'eval_steps_per_second': 0.985}

In [17]:
trainer.train()

***** Running training *****
  Num examples = 563
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 563
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss
50,4.994300
100,4.714200
150,4.413100
200,4.471100
250,4.464800
300,4.432800
350,4.380800
400,4.409400
450,4.469400
500,4.551100


Saving model checkpoint to results\checkpoint-500
Configuration saved in results\checkpoint-500\config.json
Model weights saved in results\checkpoint-500\pytorch_model.bin
tokenizer config file saved in results\checkpoint-500\tokenizer_config.json
Special tokens file saved in results\checkpoint-500\special_tokens_map.json
Deleting older checkpoint [results\checkpoint-11000] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=563, training_loss=4.521804762140684, metrics={'train_runtime': 3713.0015, 'train_samples_per_second': 0.152, 'train_steps_per_second': 0.152, 'total_flos': 305023514836992.0, 'train_loss': 4.521804762140684, 'epoch': 1.0})

In [18]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 563
  Batch size = 1


{'eval_loss': 3.5758073329925537,
 'eval_rouge1': 33.2,
 'eval_rouge2': 12.4,
 'eval_rougeL': 23.4,
 'eval_rougeLsum': 27.4,
 'eval_gen_len': 69.7,
 'eval_runtime': 7111.3982,
 'eval_samples_per_second': 0.079,
 'eval_steps_per_second': 0.079,
 'epoch': 1.0}

## MBART on Newshead summaries

In [30]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 545
  Batch size = 1


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: mhakan20. Use `wandb login --relogin` to force relogin


{'eval_loss': 4.711479663848877,
 'eval_rouge1': 28.3,
 'eval_rouge2': 17.2,
 'eval_rougeL': 25.1,
 'eval_rougeLsum': 25.3,
 'eval_gen_len': 36.9,
 'eval_runtime': 546.5747,
 'eval_samples_per_second': 0.997,
 'eval_steps_per_second': 0.997}

In [20]:
trainer.train()

C:\Users\pc\miniconda3\envs\tf\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 545
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 545
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: mhakan20. Use `wandb login --relogin` to force relogin


Step,Training Loss
50,4.190100
100,3.810800
150,4.224000
200,3.915400
250,3.922400
300,3.822600
350,3.767300
400,3.996600
450,4.130100
500,3.856800


Saving model checkpoint to results\checkpoint-500
Configuration saved in results\checkpoint-500\config.json
Model weights saved in results\checkpoint-500\pytorch_model.bin
tokenizer config file saved in results\checkpoint-500\tokenizer_config.json
Special tokens file saved in results\checkpoint-500\special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=545, training_loss=3.95895069367295, metrics={'train_runtime': 3565.6985, 'train_samples_per_second': 0.153, 'train_steps_per_second': 0.153, 'total_flos': 295271430881280.0, 'train_loss': 3.95895069367295, 'epoch': 1.0})

In [21]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 545
  Batch size = 1


{'eval_loss': 3.1031317710876465,
 'eval_rouge1': 32.4,
 'eval_rouge2': 20.4,
 'eval_rougeL': 28.8,
 'eval_rougeLsum': 29.2,
 'eval_gen_len': 35.2,
 'eval_runtime': 3964.7839,
 'eval_samples_per_second': 0.137,
 'eval_steps_per_second': 0.137,
 'epoch': 1.0}

## MBART on Newshead titles

In [19]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 545
  Batch size = 1


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: mhakan20. Use `wandb login --relogin` to force relogin


{'eval_loss': 5.749789237976074,
 'eval_rouge1': 19.1,
 'eval_rouge2': 6.4,
 'eval_rougeL': 16.8,
 'eval_rougeLsum': 17.0,
 'eval_gen_len': 35.0,
 'eval_runtime': 501.2462,
 'eval_samples_per_second': 1.087,
 'eval_steps_per_second': 1.087}

In [31]:
trainer.train()

C:\Users\pc\miniconda3\envs\tf\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 545
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 545
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss
50,5.210700
100,4.793800
150,4.648400
200,4.684800
250,4.770600
300,4.654300
350,4.802800
400,4.730800
450,4.748500
500,4.997800


Saving model checkpoint to results\checkpoint-500
Configuration saved in results\checkpoint-500\config.json
Model weights saved in results\checkpoint-500\pytorch_model.bin
tokenizer config file saved in results\checkpoint-500\tokenizer_config.json
Special tokens file saved in results\checkpoint-500\special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=545, training_loss=4.823710338347548, metrics={'train_runtime': 3619.4511, 'train_samples_per_second': 0.151, 'train_steps_per_second': 0.151, 'total_flos': 295271430881280.0, 'train_loss': 4.823710338347548, 'epoch': 1.0})

In [32]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 545
  Batch size = 1


{'eval_loss': 3.580254077911377,
 'eval_rouge1': 30.3,
 'eval_rouge2': 15.8,
 'eval_rougeL': 27.6,
 'eval_rougeLsum': 27.6,
 'eval_gen_len': 15.5,
 'eval_runtime': 2144.6005,
 'eval_samples_per_second': 0.254,
 'eval_steps_per_second': 0.254,
 'epoch': 1.0}